**视频地址**
+ <https://www.bilibili.com/video/BV1t44y1r7ct?p=2>

**jupyter notebook**
+ <https://courses.d2l.ai/zh-v2/assets/notebooks/chapter_convolutional-neural-networks/lenet.slides.html#/>

LeNet(LeNet-5)由两个部分组成，卷积编码器和全连接dense层

In [1]:
import torch
from torch import nn
import d2l_torch as d2l

import os
os.environ["OMP_NUM_THREADS"] = "1"

关于x.view()函数，其实就是对x的浅拷贝，简单来说，就是操作直接加在x上，原地操作。不会产生新的内存空间

# 网络定义

In [3]:
class Reshape(torch.nn.Module):
    def forward(self,x):
        return x.view(-1,1,28,28)   # 这里相当于把输入的x变成 (batch_size,输入通道，长，宽)，其中batch_size是根据其他几项自动计算的

    
net=torch.nn.Sequential(
    Reshape(),nn.Conv2d(1,6,kernel_size=5,padding=2),nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2,stride=2),
    # 这里使用的是平均池化
    nn.Conv2d(6,16,kernel_size=5),nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2,stride=2),nn.Flatten(),
    nn.Linear(16*5*5,120),nn.Sigmoid(),
    nn.Linear(120,84),nn.Sigmoid(),
    nn.Linear(84,10))

+ 注意，关于原始图像到底是32还是28，其实不是很重要的，有的是提前padding了，有的是在卷积层才padding。反正大致就是这个意思，实现下来都一样
+ 关于sigmoid函数，如果忘了可以参考：[动手学深度学习V2.0(Pytorch)——10.感知机](https://blog.csdn.net/Castlehe/article/details/120523671)部分搜索sigmoid函数
+ nn.Flatten()，默认是保持第一个维度（最外层的中括号，从左到右数第一个中括号），剩下的拉平
+ pytorch不那么好的地方——Linear层的输入输出需要自己算（自己指明），但是像keras和mxnet这种symbolic的框架就不用算，自动填好的
+ 这里没有在最后一个全连接层加入激活函数，是因为pytorch的交叉熵损失，默认是整合了softmax函数的，所以这里没有在最后一层全连接层加入softmax激活函数

# 检查模型

## 打印模型输出维度信息

In [8]:
net_len=len(net)

In [16]:
# 随便生成一个输入batch看下模型情况
X=torch.rand((1,1,28,28),dtype=torch.float32)
for i,layer in zip(range(net_len),net):
    X=layer(X)
    print(f"{i}.",layer.__class__.__name__,f"output_shape,{X.shape}")
    
# 如果全连接层输入不会算，就直接以这种方式，run一下，然后自己就不用算了。。就可以看到上一个池化层输出的维度，然后就可以自己填进去了

0. Reshape output_shape,torch.Size([1, 1, 28, 28])
1. Conv2d output_shape,torch.Size([1, 6, 28, 28])
2. Sigmoid output_shape,torch.Size([1, 6, 28, 28])
3. AvgPool2d output_shape,torch.Size([1, 6, 14, 14])
4. Conv2d output_shape,torch.Size([1, 16, 10, 10])
5. Sigmoid output_shape,torch.Size([1, 16, 10, 10])
6. AvgPool2d output_shape,torch.Size([1, 16, 5, 5])
7. Flatten output_shape,torch.Size([1, 400])
8. Linear output_shape,torch.Size([1, 120])
9. Sigmoid output_shape,torch.Size([1, 120])
10. Linear output_shape,torch.Size([1, 84])
11. Sigmoid output_shape,torch.Size([1, 84])
12. Linear output_shape,torch.Size([1, 10])


+ 其实0、1、2、3这四部分共同属于一个功能模块，通道从1变成6，高宽减半，所以实际上信息是变多了
+ 4、5、6这三个属于一个组，这一整片的输入其实是(6,14,14)，输出其实是(16,5,5)。也都是通道数增加，长宽减小
+ 然后经过7压平
+ 剩下的几个全连接层其实就是多层感知机（MLP），就是希望通过hidden_size把这个输入的维度逐渐平滑（smooth）的往下降

----
这里卷积层和卷积的激活函数分别作为两层，但是其实可以是一层的，不是很重要。重点是看维度变化。
+ 可以发现，输入其实就是$28\times 28$，经过`2.`的卷积层以及激活函数之后，由于加入了padding，所以即便卷积核是5，也是$28+2\times 2-5+1$=28
+ 然后`4.`这个池化层，其实就是（28+2-2）/2=14，所以输出维度就是14
+ 然后`5.`这个卷积层，就是14-5+1=10,所以输出就是10
+ `7.`这个池化层，就是（10+2-2）/2=5
+ [1,7]层输出的形状，都是(batch_size,channel,length,width)
+ 到了`9.` 其实输入就是16个通道，每个通道的feature map是$5\times 5$，所以就是$16\times 5\times 5=400$，输出大小是自己定的，120（虽然不知道这样的依据是什么，可以自己试验感受一下）
+ `11.`和`13.`同理，

## 模型构造解释

+ 之前在[多输入输出通道中](https://blog.csdn.net/Castlehe/article/details/121844745)讲过
+ 卷积核得到的输出，其实就是识别某种特定的模式，
    + 可以认为，每个输出通道都对应一种特定的模式。即：可以通过学到的卷积核的参数，来匹配出某一个特定的模式。
    + 因此多输出通道的作用就是：每个输出通道都会识别出一个特定的模式
    + 多输入通道的作用就是：输入通道核识别并组合输入中的模式，得到一个组合的模式识别。
+ 因此大部分卷积层，都在不断增大通道数，减小每个通道的大小。这样就可以识别更多的模式
    + 因此，Lenet的卷积部分，也是不断增加输出通道（增加可以识别的模式），减小输出通道的大小
+ 而多层感知机其实就是为了把这部分内容平缓进行压缩，得到最终的分类结果

## 打印模型信息

除了自己直接打印每层，其实也可以直接使用summary函数

+ 如果希望打印的好看一点，那么可以考虑安装一下这个`torchsummary`这个库，然后summary一下。
+ 但是实际上，也不是很有必要，毕竟直接print(model)，也可以啊
+ 参考：[Model summary in pytorch](https://stackoverflow.com/questions/42480111/model-summary-in-pytorch)


```python
    from torchsummary import summary
    summary(net, (3, 224, 224))
```

In [15]:
print(net)

Sequential(
  (0): Reshape()
  (1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (2): Sigmoid()
  (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (5): Sigmoid()
  (6): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (7): Flatten(start_dim=1, end_dim=-1)
  (8): Linear(in_features=400, out_features=120, bias=True)
  (9): Sigmoid()
  (10): Linear(in_features=120, out_features=84, bias=True)
  (11): Sigmoid()
  (12): Linear(in_features=84, out_features=10, bias=True)
)


# 查看Lenet在FASHION_MNIST数据集上的表现

因为MNIST基本就是99%，没什么好看的，所以fashion-mnist稍微难一点，所以看这个

In [17]:
batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size)

对之前的`evalute_accuracy`函数略微进行修改，添加了对device的判断

In [18]:
def evaluate_accuracy_gpu(net, data_iter, device=None):  
    """使用GPU计算模型在数据集上的精度。"""
    if isinstance(net, torch.nn.Module):
        net.eval()
        if not device:
            device = next(iter(net.parameters())).device #根据待训练参数存储的位置，来决定使用哪个设备（参数存在cpu，就cpu；GPU，就gpu）
    metric = d2l.Accumulator(2)
    for X, y in data_iter:
        if isinstance(X, list):
            X = [x.to(device) for x in X]
            # 如果X是个列表，那就每次挪一个，把数据挪到选择的device上去
        else:
            X = X.to(device)
            # 如果不是列表，只是一个tensor，那就一次挪完
        y = y.to(device)
        metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [19]:
iter(net.parameters())  # 这个其实就是得到一个生成器

<generator object Module.parameters at 0x7fdbe3e5a6d0>

In [22]:
next(iter(net.parameters()))  # 每次调用，取生成器中的一个对象
# next(iter(net.parameters())).shape # 这个就是第一个卷积层的6个$5\times 5$的卷积核

Parameter containing:
tensor([[[[-0.0650,  0.0876,  0.1100,  0.0343,  0.1212],
          [-0.1999,  0.0912,  0.1086, -0.0905, -0.1887],
          [ 0.1587, -0.1790, -0.1672, -0.1618,  0.1311],
          [-0.1913,  0.1998,  0.0584,  0.1044,  0.1000],
          [-0.0438,  0.1223, -0.0231, -0.0776,  0.0903]]],


        [[[-0.0581,  0.1834,  0.0053, -0.0773,  0.1098],
          [ 0.0668, -0.0665,  0.0757,  0.1585,  0.0179],
          [ 0.1062, -0.1994,  0.1144, -0.0615,  0.1532],
          [-0.0960,  0.1843, -0.1660,  0.0306, -0.0858],
          [ 0.1081, -0.0633,  0.0248, -0.0779, -0.0617]]],


        [[[-0.1948,  0.0438,  0.1628, -0.1689, -0.0051],
          [-0.0694, -0.0092,  0.0039, -0.1828, -0.0110],
          [ 0.1665,  0.1355,  0.1803,  0.1952, -0.0552],
          [-0.1657,  0.0524,  0.1514, -0.1434,  0.1830],
          [-0.1144, -0.1332,  0.0438, -0.1473, -0.0429]]],


        [[[-0.0550, -0.0142, -0.1250,  0.1094,  0.1550],
          [ 0.0086, -0.0705, -0.0678, -0.1256, -0.1304

13：04